# Anthropogenic heat from human metabolism

This notebook is used to process population data, and ultimately produce the anthropogenic heat flux (AHF) from human metabolism for the Greater Los Angeles (LA) region.

In [ ]:
# load packages 
%matplotlib inline
import os
import glob
from pathlib import Path
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
#import requests
import geopandas as gpd
import pyproj
import shapely.geometry
import contextily as ctx
from matplotlib.ticker import FormatStrFormatter
import geofeather
import dask_geopandas as dgpd
import pyogrio
import folium
import numpy as np